# Creating Message Passing Networks
PYG 提供了MessagePassing的基类，会自己进行信息传递，用户需要定义phi、gamma函数和汇聚模式（sum、mean、max）

```python
MessagePassing(aggr="add", flow="source_to_target", node_dim=-2)
```

以下方法让我们能够进行网络的创建
1. MessagePassing(aggr="add",flow=source_to_target",node_dim=-2)

  node_dim : 哪个维度的数据被传播？

2. MessagePassing.propagate(edge_index,size=None,**kwargs)

进行消息传播的初始调用，将节点的信息和边的信息收集起来，然后进行节点向量的更新

3. MessagePassing.message(...) 构建节点i的信息，参数是通过propagate的参数。

4. MessagePassing.update(aggr_out,...),更新所有的节点的嵌入向量

# IMPLEMETING THE GCN LAYER
